In [ ]:
from sklearn.metrics import mean_squared_error

from sklearn.metrics import mean_absolute_error as MAE

from sklearn.preprocessing import OneHotEncoder

from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb
from sklearn.preprocessing import StandardScaler,MaxAbsScaler,MinMaxScaler,OrdinalEncoder
from sklearn.model_selection import GridSearchCV
#%pip install catboost
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%pip install catboost
import catboost as cb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd "drive/MyDrive/Colab Notebooks/Challenge/"

[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/Challenge/'
/content/drive/MyDrive/Colab Notebooks/Challenge


In [ ]:

train_station = pd.read_csv("remCharEWM4.csv", sep=",")
train_station['date'] = pd.to_datetime(train_station['date'])
train_station['Postcode'] = train_station['Postcode'].astype(str)


train_global = train_station.groupby('date').agg({'Available': 'sum',
                                                  'Charging': 'sum',
                                                  'Passive': 'sum',
                                                  'Other': 'sum',
                                                  'tod': 'max',
                                                  'dow': 'max',
                                                  'trend': 'max'}).reset_index()

test_station=pd.read_csv("test.csv", sep=",")
test_station['date'] = pd.to_datetime(test_station['date'])
test_station['Postcode'] = test_station['Postcode'].astype(str)
test_global= test_station.groupby('date').agg({
    'tod': 'max',
    'dow': 'max',
    'trend': 'max'}).reset_index()


In [ ]:
def make_cols(data,not_test=True):
  if not_test :
    data["labels"]=data.Available.astype(int).astype(str) + data.Charging.astype(int).astype(str) + data.Passive.astype(int).astype(str)+ data.Other.astype(int).astype(str)
  data['hour'] = data.date.dt.hour
  data['min'] = data.date.dt.minute
  data['quarter'] = data.date.dt.quarter
  data['dayofmonth'] = data.date.dt.day
  return data


train_global=make_cols(train_station)
test_global=make_cols(test_station,False)
df_train=train_station.copy().loc[train_station.date < '2021-02-1 00:00:00']
df_val=train_station.copy().loc[train_station.date >= '2021-02-1 00:00:00']


In [ ]:

col_act=["hour","tod","min","dow","trend"]
targets=["Available"	,"Charging"	,"Passive",	"Other"	]
cat_x_train= train_station[col_act].copy()
cat_y_train=train_station[targets].copy()
cat_x_test=df_val[col_act].copy()
cat_y_test=df_val[targets].copy()


In [ ]:

cat_features_index = [0,3]


dec_values={'depth': 3, 'iterations': 1500, 'l2_leaf_reg': 1, 'learning_rate': .1}
#print(cb_model.best_params_,cb_model.best_score_)
clf = cb.CatBoostRegressor(eval_metric="MultiRMSE",loss_function="MultiRMSE",one_hot_max_size=200,depth= dec_values["depth"],
                          iterations = dec_values["iterations"], learning_rate= dec_values["learning_rate"],
                           l2_leaf_reg= dec_values["l2_leaf_reg"])# 0.1
chain=(clf).fit(cat_x_train,cat_y_train, cat_features= cat_features_index,verbose=100)
# {'depth': 5, 'iterations': 1000, 'l2_leaf_reg': 1, 'learning_rate': 0.01}
# 
chain

0:	learn: 1.9204067	total: 410ms	remaining: 10m 13s
100:	learn: 1.8999912	total: 38.4s	remaining: 8m 52s
200:	learn: 1.8981145	total: 1m 17s	remaining: 8m 23s
300:	learn: 1.8972564	total: 1m 54s	remaining: 7m 37s
400:	learn: 1.8967499	total: 2m 31s	remaining: 6m 54s
500:	learn: 1.8964062	total: 3m 7s	remaining: 6m 13s
600:	learn: 1.8961761	total: 3m 43s	remaining: 5m 34s
700:	learn: 1.8959820	total: 4m 20s	remaining: 4m 56s
800:	learn: 1.8958500	total: 4m 56s	remaining: 4m 18s
900:	learn: 1.8957247	total: 5m 32s	remaining: 3m 40s
1000:	learn: 1.8956204	total: 6m 8s	remaining: 3m 3s
1100:	learn: 1.8955439	total: 6m 44s	remaining: 2m 26s
1200:	learn: 1.8954735	total: 7m 19s	remaining: 1m 49s
1300:	learn: 1.8954219	total: 7m 55s	remaining: 1m 12s
1400:	learn: 1.8953737	total: 8m 34s	remaining: 36.3s
1499:	learn: 1.8953270	total: 9m 9s	remaining: 0us


In [ ]:
y_pred_class=np.floor(#cb_model.best_estimator_.predict(cat_x_test)).astype(int)
                     chain.predict(cat_x_test)).astype(int)

df55=df_val[["Available","Charging","Passive","Other"]].copy()
df55[[i+"_" for i in targets]]=y_pred_class

mea00=MAE(df55[[i+"_" for i in targets]],df55[targets],multioutput="raw_values")

print(f'sklearn MAE \n{mea00}\n')
print(f'\n{mea00.mean()}\n\n')
print(f'sklearn MAE on the mean \n\n{100*mea00/df55[targets].mean(axis=0)}%')
#[5.47350898 2.10856977 1.41632889 8.37072959]



#sklearn MAE on the mean 
# 6.50347 #4.829137731481481
#sklearn MAE on the mean  #6.051359953703704


#[17.98726852  4.78472222  3.45023148 29.07002315]
#13.823061342592592 13.302083333333332 #6.686921296296297 12.660 10.17
#sklearn MAE on the mean 
#Available    13.727586
#Charging     18.405236
#Passive      21.208025
#Other        42.381061
#13.9921875

#12.233362268518519
#12.070601851851851
#11.719039351851851

sklearn MAE 
[4.24937561 3.22309412 2.22506611 3.07021183]


3.1919369175673418


sklearn MAE on the mean 

Available     3.167597
Charging     12.303050
Passive      13.440409
Other         3.194941
dtype: float64%


In [ ]:
df333=test_station.copy()
test_pred_class=np.floor(chain.predict(test_station[col_act])).astype(int)
df333[targets]=test_pred_class
df333

,date,tod,dow,trend,hour,min,dayofmonth,Available,Charging,Passive,Other
0,2021-02-19 00:00:00,0,6,22203,0,0,19,135,24,11,101
1,2021-02-19 00:15:00,1,6,22204,0,15,19,135,24,11,101
2,2021-02-19 00:30:00,2,6,22205,0,30,19,135,24,12,101
3,2021-02-19 00:45:00,3,6,22206,0,45,19,135,23,12,101
4,2021-02-19 01:00:00,4,6,22207,1,0,19,135,23,12,101
...,...,...,...,...,...,...,...,...,...,...,...
1819,2021-03-10 22:45:00,91,4,24118,22,45,10,137,23,11,101
1820,2021-03-10 23:00:00,92,4,24119,23,0,10,137,22,11,101
1821,2021-03-10 23:15:00,93,4,24120,23,15,10,138,21,11,101
1822,2021-03-10 23:30:00,94,4,24121,23,30,10,138,21,12,101


In [ ]:
df333[['date'] +
                  targets].to_csv("global.csv", index=False)
